# <img src="../img/divvy_logo.svg" class="img-header"> command-line tutorial

`Divvy` also provides a command-line interface that gives you the same power as the python API. You can use `--help` to get a list of the command-line options:

In [29]:
divvy --help

version: 0.2-dev
usage: divvy [-h] [-V] [-C CONFIG] [-S SETTINGS] [-P PACKAGE] -O OUTFILE

divvy - write compute jobs that can be submitted to any computing resource

optional arguments:
  -h, --help            show this help message and exit
  -V, --version         show program's version number and exit
  -C CONFIG, --config CONFIG
                        Divvy configuration file.
  -S SETTINGS, --settings SETTINGS
                        YAML file with job settings to populate the template.
  -P PACKAGE, --package PACKAGE
                        Compute package
  -O OUTFILE, --outfile OUTFILE
                        Output filepath

https://github.com/pepkit/divvy


## The settings file

The settings argument is where you can pass an existing `yaml` file with key-value pairs. Here's a simple example:

In [30]:
cat settings.yaml

time: 4-0-0
logfile: results.log
cores: 6
partition: large_mem
mem: 16G


Now let's take a look at the template we are going to use by activating the `slurm` package

In [31]:
cat ../divvy/submit_templates/slurm_template.sub

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='{TIME}'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE}


We use `divvy` to populate that template with our list of variables above, like this:

In [32]:
divvy -P slurm -S settings.yaml -O test.sub

Using default config file, no global config file provided in environment variable(s): ['DIVCFG', 'PEPENV']
Loading divvy config file: /home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/default_compute_settings.yaml
Available packages: default, local, slurm
Activating compute package 'default'
Activating compute package 'slurm'
Loading yaml settings file: settings.yaml
Writing script to /home/nsheff/code/divvy/docs_jupyter/test.sub


Now we can take a look at what our sbumission scripts looks like.

In [33]:
cat test.sub

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='results.log'
#SBATCH --mem='16G'
#SBATCH --cpus-per-task='6'
#SBATCH --time='4-0-0'
#SBATCH --partition='large_mem'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE}


We populated several variables, like `{LOGFILE}` and `{TIME}`, from the `settings.yaml` file. However, the `{CODE}` and `{JOBNAME}` variables are still unpopulated, so this submission script is incomplete. To remedy this, we'll use `divvy`'s command-line variable passing: any non-interpreted arguments passed to `divvy` are assumed to be variables to populate the template. These command-line variables are considered highest priority and so will override any values in the more distant locations. For example:

In [34]:
divvy -P slurm -S settings.yaml -O test.sub --code run-this-cmd --jobname 12345 --time 6-0-0

Custom vars: {}
Using default config file, no global config file provided in environment variable(s): ['DIVCFG', 'PEPENV']
Loading divvy config file: /home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/default_compute_settings.yaml
Available packages: default, local, slurm
Activating compute package 'default'
Activating compute package 'slurm'
Loading yaml settings file: settings.yaml
Writing script to /home/nsheff/code/divvy/docs_jupyter/test.sub


In [35]:
cat test.sub

#!/bin/bash
#SBATCH --job-name='12345'
#SBATCH --output='results.log'
#SBATCH --mem='16G'
#SBATCH --cpus-per-task='6'
#SBATCH --time='6-0-0'
#SBATCH --partition='large_mem'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

run-this-cmd


Now we have a complete script, which we can run with `sbatch test.sub`. Notice also that the `time` variable uses the one provided on the CLI rather than the one provided in the `settings.yaml` file, because the CLI has a higher priority.

Variables can come from these 3 sources, in order of increasing priority: 1) compute package (defined in the `divvy` configuration file and selected with the `-P` or `--package` argument); 2) `settings.yaml` file, passed with `-S` or `--settings`; 3) any additional variables passed on the command line.